In [2]:
## 여러 입력 값을 갖는 로지스틱 회귀 실제 값 적용하기

#7시간 공부하고 과외를 6번 받은 학생의 합격 가능성

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

#실행할 때마다 같은 결과를 출력하기 위한 seed값 설정
seed = 0
np.random.seed(seed)#넘파이, 텐서플로우 seed값 초기화
tf.set_random_seed(seed)

#x, y의 데이터 값
x_data = np.array([[2,3],[4,3],[6,4],[8,6],[10,7],[12,8],[14,19]])#[공부시간, 과외횟수]
y_data = np.array([0,0,0,1,1,1,1]).reshape(7,1)#가로 7, 세로1로 만든다. 합격/불합격 결과

#입력 값을 플레이스 홀더에 저장
X = tf.placeholder(tf.float64, shape=[None, 2])#앞은 상관없고, 1차원 요소는 2개
Y = tf.placeholder(tf.float64, shape=[None, 1])#앞은 상관없고, 1차원 요소는 1개

#기울기 a와 바이어스 b의 값을 임의로 정함
#a는 [2,1]이 형태를 가짐 a1, a2
a = tf.Variable(tf.random_uniform([2,1], dtype=tf.float64))#기울기 2개니까 a1, a2 묶어서 [2,1](하나로 표현된 식이 2개 있다)
#[2,1] 의미 : 들어오는 값은 2개, 나가는 값은 1개
b = tf.Variable(tf.random_uniform([1], dtype=tf.float64))

#y 시그모이드 함수의 방정식을 세움
y = tf.sigmoid(tf.matmul(X, a) + b)    #예측치. () 안 부분은 -z식

#오차를 구하는 함수
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y))

#학습률 값
learning_rate = 0.1

#오차를 최소로 하는 값 찾기
gradient_decent = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

predicted = tf.cast(y > 0.5, dtype=tf.float64) #y가 0.5보다 크면 1, 아니면 0(텐서플로우의 cast연산자에 의해)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float64))#텐서플로우의 평균(predicted와 Y가 같으면 1, 아니면 0인 값들의 평균)

#학습
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(3001):
        a_, b_, loss_, _ = sess.run([a, b, loss, gradient_decent], feed_dict={X:x_data, Y:y_data})#플레이스홀더가 있으면 feed_dict로 값을 넣어준다
        if (i+1) % 300 == 0:
            print("step = %d, a1 = %.4f, a2 = %.4f, b = %.4f, loss = %.4f" %(i+1, a_[0], a_[1], b_, loss_))
    
    
    print()
    #추가 코드(sessoin안에 있어야 함)
    print("predicted = ", sess.run(predicted, feed_dict={X:x_data})) #
    #다른 값 테스트
    p_val, h_val = sess.run([predicted, y], feed_dict={X:[[1, 5], [10, 5], [4, 5]]})
    print("check predicted = ", p_val)    #연산된 값을 bool로 변환한 값
    print("check hypothesis = ", h_val)   #계산값
    #정확도 측정
    h, c, a = sess.run([y, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis : ", h, "\nCorrect (Y) :", c, "\nAccuracy : ",a)#예측치, 결과, 정확도 인듯....
    
    #===========================================여기부터 오늘 코드=============================================
    new_x = np.array([7,6]).reshape(1,2) #7,6은 각각 공부한 시간과 과외수업횟수
    new_y, new_y_result = sess.run([y, predicted], feed_dict={X:new_x})#새 합격 가능성, 그걸 predicted한게 y_result, 데이터를 플레이스 홀더에 넣어준다
    
    print("공부한 시간 : %d, 과외 수업 횟수 : %d" %(new_x[:,0], new_x[:,1]))
    print("합격 가능성 : %6.2f %%" %(new_y*100))    #%하나만 쓰면 %글자 출력 안되니까 %%쓴거다
    print("예측 결과 :",new_y_result[0])

step = 300, a1 = 0.3977, a2 = 0.0820, b = -2.7195, loss = 0.2517
step = 600, a1 = 0.5033, a2 = 0.2126, b = -4.2252, loss = 0.1733
step = 900, a1 = 0.4990, a2 = 0.4149, b = -5.2534, loss = 0.1364
step = 1200, a1 = 0.4515, a2 = 0.6366, b = -6.0584, loss = 0.1130
step = 1500, a1 = 0.3914, a2 = 0.8503, b = -6.7260, loss = 0.0964
step = 1800, a1 = 0.3300, a2 = 1.0476, b = -7.2984, loss = 0.0841
step = 2100, a1 = 0.2715, a2 = 1.2274, b = -7.7999, loss = 0.0745
step = 2400, a1 = 0.2172, a2 = 1.3908, b = -8.2467, loss = 0.0668
step = 2700, a1 = 0.1673, a2 = 1.5395, b = -8.6496, loss = 0.0606
step = 3000, a1 = 0.1217, a2 = 1.6754, b = -9.0167, loss = 0.0554

predicted =  [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]]
check predicted =  [[0.]
 [1.]
 [0.]]
check hypothesis =  [[0.37349032]
 [0.64036528]
 [0.4619409 ]]

Hypothesis :  [[0.02303809]
 [0.02919472]
 [0.17006392]
 [0.88180058]
 [0.98070748]
 [0.99712126]
 [1.        ]] 
Correct (Y) : [[0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]] 
Accuracy :  

In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

seed = 0
np.random.seed(seed)
tf.set_random_seed(seed)

#data = np.loadtxt("C:/Users/Owner/Desktop/khh/Likelion/211008/data-03-diabetes.csv", delimiter=",")
#x_data = [[i[:-1]] for i in data] # 0 ~ 7번까지의 값이 x_data
#y_data = [i[8] for i in data] #마지막 8번째 값이 y_data

#배열 슬라이싱 응용
xy = np.loadtxt("/content/data-03-diabetes.csv", delimiter=",")
x_data = xy[:, 0:-1]    #[전체줄, 각 줄 데이터 중 0~맨 끝에거 앞에까지]
y_data = xy[:, [-1]]    #[전체줄, 각 줄 데이터 중 맨 끝에거]

#Placeholders : Shape주의! 총 8개의 x_data와 1개의 y_data
X = tf.placeholder(tf.float32, shape=[None,8])#
Y = tf.placeholder(tf.float32, shape=[None,1])#

W = tf.Variable(tf.random_normal([8,1]), name='weight')#기울기 8개? x값이 8개라 우선 8로 잡아봄
b = tf.Variable(tf.random_normal([1]), name='bias')
#random_normal은 어느정도 예측할 수 있을때 그 값을 중심으로 하지만
#대부분은 예측할 수 없으미까 uniform을 쓰는게 좋다

#Hypothesis예측값
hyp = tf.sigmoid(tf.matmul(X, W) + b)

#Cost/Loss function
cost = -tf.reduce_mean(Y * tf.log(hyp) + (1 - Y) * tf.log(1-hyp))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

#정확도 hyp > 0.5 else False
predicted = tf.cast(hyp > 0.5, dtype=tf.float32)#
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))#

#세션 시작
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        cost_val, _ = sess.run([cost, train], feed_dict={X:x_data, Y:y_data})
        if step % 200 == 0:
            print(step, cost_val)
            # 10000 0.480384
        
    #정확도 77%
    _, _, a = sess.run([hyp, predicted, accuracy], feed_dict={X:x_data, Y:y_data})#acc앞은 연산을 위해 필요한거니까 출력 안해도 됨
    #a = sess.run(accuracy, feed_dict={X:x_data, Y:y_data})로 해도 문제는 안됨
    #graph가 구성되어 있으므로
    print("Accuracy :", a)
    #Accuracy : 0.769433

0 0.9754207
200 0.78652614
400 0.73401636
600 0.7038551
800 0.6796619
1000 0.6588307
1200 0.6406826
1400 0.6248361
1600 0.6109787
1800 0.59883696
2000 0.58817154
2200 0.57877535
2400 0.57047045
2600 0.5631049
2800 0.5565494
3000 0.5506938
3200 0.54544497
3400 0.5407234
3600 0.53646153
3800 0.53260183
4000 0.5290952
4200 0.5258994
4400 0.5229783
4600 0.5203007
4800 0.5178397
5000 0.51557213
5200 0.51347744
5400 0.51153815
5600 0.5097386
5800 0.50806534
6000 0.5065064
6200 0.5050512
6400 0.50369036
6600 0.5024156
6800 0.5012195
7000 0.50009567
7200 0.49903786
7400 0.4980409
7600 0.49710017
7800 0.4962113
8000 0.49537027
8200 0.49457365
8400 0.4938183
8600 0.4931012
8800 0.4924198
9000 0.49177158
9200 0.4911545
9400 0.49056628
9600 0.49000522
9800 0.48946962
10000 0.48895785
Accuracy : 0.76679844


In [9]:
# 퍼셉트론
#AND, OR, NAND를 구현하는 로직연산
#퍼셉트론은 XOR못한다

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
#y_data = np.array([[0],[0],[0],[1]], dtype=np.float32)#And를 구현하는 로직
#y_data = np.array([[0],[1],[1],[1]], dtype=np.float32)#OR를 구현하는 로직
y_data = np.array([[1],[1],[1],[0]], dtype=np.float32)#NAND를 구현하는 로직

X = tf.placeholder(tf.float32, shape=[None,2])
Y = tf.placeholder(tf.float32, shape=[None,1])

W = tf.Variable(tf.random_normal([2,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hyp = tf.sigmoid(tf.matmul(X, W) + b)

cost = -tf.reduce_mean(Y * tf.log(hyp) + (1 - Y) * tf.log(1-hyp))   #오차
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

predicted = tf.cast(hyp > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 1000 == 0:
            print("step =", step, "cost =", sess.run(cost, feed_dict={X:x_data, Y:y_data}))
            print('W =',sess.run(W), "b =",sess.run(b))

    h, p, a = sess.run([hyp, predicted, accuracy], feed_dict={X:x_data, Y:y_data})
    print("\nHypothesis :",h,"\nCorrect",p,"\nAccuracy :", a)
  
    
    #실제값 확인
    print("\n다른값 테스트")
    new_x = np.array([0,1]).reshape(1,2)
    new_y = sess.run(hyp, feed_dict={X:new_x})
    h, c = sess.run([hyp, predicted], feed_dict={X:new_x, Y:new_y})
    print('Hyothesis :', h, "\nCorrect(Y) :",c)

step = 0 cost = 0.9453583
W = [[1.0201889]
 [0.5112581]] b = [-0.8465425]
step = 1000 cost = 0.5874238
W = [[ 0.15150224]
 [-0.1303716 ]] b = [0.6277868]
step = 2000 cost = 0.4319353
W = [[-0.5690266]
 [-0.7315243]] b = [1.4407197]
step = 3000 cost = 0.34515724
W = [[-1.0851076]
 [-1.1815641]] b = [2.0686817]
step = 4000 cost = 0.28975335
W = [[-1.4823263]
 [-1.541385 ]] b = [2.583518]
step = 5000 cost = 0.2508015
W = [[-1.8067298]
 [-1.8440031]] b = [3.0216131]
step = 6000 cost = 0.22159007
W = [[-2.082617 ]
 [-2.1068184]] b = [3.4044037]
step = 7000 cost = 0.19869198
W = [[-2.3238504]
 [-2.3399785]] b = [3.7452745]
step = 8000 cost = 0.18016587
W = [[-2.538957 ]
 [-2.5499647]] b = [4.0530934]
step = 9000 cost = 0.16481918
W = [[-2.7335358]
 [-2.7412126]] b = [4.3340473]
step = 10000 cost = 0.15187196
W = [[-2.9114633]
 [-2.9169228]] b = [4.5926466]

Hypothesis : [[0.98997545]
 [0.8423375 ]
 [0.8430612 ]
 [0.22517842]] 
Correct [[1.]
 [1.]
 [1.]
 [0.]] 
Accuracy : 1.0

다른값 테스트
Hyothes